# t-SNE Animations
`openTSNE` includes a callback system, with can be triggered every `n` iterations and can also be used to control optimization and when to stop.

In this notebook, we'll look at an example and use callbacks to generate an animation of the optimization. In practice, this serves no real purpose other than being fun to look at.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd 
import numpy as np

from openTSNE import TSNE
from sklearn.preprocessing import OrdinalEncoder

# Load Data

In [ ]:
df = pd.read_csv("../data/loan_approval.csv")

X = df.drop('loan_status', axis=1)
y = df['loan_status']

cat_cols = X.select_dtypes(include=['object', 'category']).columns

encoder = OrdinalEncoder()
X[cat_cols] = encoder.fit_transform(X[cat_cols])

X = np.array(X)
y = np.array(y)

In [ ]:
embeddings = []

tsne = TSNE(
    n_jobs=32,
     # The embedding will be appended to the list we defined above, make sure we copy the
    # embedding, otherwise the same object reference will be stored for every iteration
    callbacks=lambda it, err, emb: embeddings.append(np.array(emb)),
    # This should be done on every iteration
    callbacks_every_iters=1,
)

tsne_embedding = tsne.fit(X)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_xticks([]), ax.set_yticks([])

color_map = {0: 'blue', 1: 'red'}
colors = list(map(color_map.get, y))
pathcol = ax.scatter(embeddings[0][:, 0], embeddings[0][:, 1], c=colors, s=1, rasterized=True)

def update(embedding, ax, pathcol):
    # Update point positions
    pathcol.set_offsets(embedding)
    
    # Adjust x/y limits so all the points are visible
    ax.set_xlim(np.min(embedding[:, 0]), np.max(embedding[:, 0]))
    ax.set_ylim(np.min(embedding[:, 1]), np.max(embedding[:, 1]))
    
    return [pathcol]

anim = animation.FuncAnimation(
    fig, update, fargs=(ax, pathcol), interval=20,
    frames=embeddings, blit=True,
)

anim.save("sample_vid.mp4", dpi=150, writer="ffmpeg")
plt.close()